In [2]:
import numpy as np
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
from skimage import io as skio
import torch
import torch.nn as nn
from efficientnet_pytorch import model as efficientnet_model
from concurrent.futures import ThreadPoolExecutor
from torch.utils.data import Dataset, DataLoader

/home/woshington/Projects/Doutorado/venv/lib/python3.10/site-packages/albumentations/check_version.py:51: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


In [3]:
dataset_dir="../dataset"
data_csv_path=f"../data"
output_dimensions = 5

In [4]:
df_train = pd.read_csv(f"{data_csv_path}/train.csv")
df_val = pd.read_csv(f"{data_csv_path}/val.csv")
df_test = pd.read_csv(f"{data_csv_path}/test.csv")
df_pen_mark = pd.read_csv(f"{data_csv_path}/without_pen_mask.csv")

df_train = df_train[~df_train["image_id"].isin(df_pen_mark["image_id"])]
df_val = df_val[~df_val["image_id"].isin(df_pen_mark["image_id"])]
df_test = df_test[~df_test["image_id"].isin(df_pen_mark["image_id"])]

IMG_DIR = f"{dataset_dir}/tiles"

In [5]:
transforms_train = A.Compose([
    A.Transpose(p=0.5),
    A.Resize(224, 224),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.ToFloat(),
    ToTensorV2()
])

In [6]:
def extract_features(args):
    image_path, label, index, transform, model = args
    print(f"Processando item: {index+1}")

    image = skio.imread(image_path)
    
    if transform:
        image = transform(image=image)["image"]
    
    image /= 255
    image = image.unsqueeze(0)
    
    with torch.no_grad():
        features = model(image)
    
    features = features.flatten(start_dim=1).squeeze().numpy()
    
    return features, label

In [7]:
class EfficientNetFeatureExtractor(nn.Module):    
    def __init__(self):
        super().__init__()
        self.efficient_net = efficientnet_model.EfficientNet.from_pretrained("efficientnet-b0")        
        self.efficient_net.load_state_dict(
            torch.load("../pre-trained-models/efficientnet-b0-08094119.pth", weights_only=True)
        )
        self.efficient_net._fc = nn.Identity()

    def extract(self, inputs):
        return self.efficient_net.extract_features(inputs)

    def forward(self, inputs):
        return self.extract(inputs)        


In [8]:
class ImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = f"{self.img_dir}/{row['image_id']}.jpg"
        label = row['isup_grade']
        
        image = skio.imread(img_path)  # Lê a imagem
        if self.transform:
            image = self.transform(image=image)["image"]
        image = image / 255.0  # Normaliza para [0, 1]
        
        return image.clone().detach().float(), label

In [9]:
def extract_features_dataloader(data_loader, model, device):
    all_features = []
    all_labels = []
    model.eval()
    model.to(device)
    
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)  # Move images para o mesmo dispositivo do modelo
            features = model(images)  # Extrai características
            features = features.view(features.size(0), -1).cpu().numpy()  # Achata para 2D
            all_features.append(features)
            all_labels.append(labels.numpy())  # Converte rótulos para NumPy
        
    all_features = np.vstack(all_features)  # Combina todas as características em um array 2D
    all_labels = np.concatenate(all_labels)  # Combina todos os rótulos em 1D
    return all_features, all_labels


In [10]:
train_dataset = ImageDataset(df_train, IMG_DIR, transform=transforms_train)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4)

val_dataset = ImageDataset(df_val, IMG_DIR, transform=transforms_train)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=4)

test_dataset = ImageDataset(df_test, IMG_DIR, transform=transforms_train)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=4)

In [11]:
model = EfficientNetFeatureExtractor()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loaded pretrained weights for efficientnet-b0


In [12]:
train_features, train_labels = extract_features_dataloader(train_loader, model, device)
train_data = np.column_stack((train_features, train_labels))

In [13]:
np.save("train_data.npy", train_data)

In [14]:
val_features, val_labels = extract_features_dataloader(val_loader, model, device)
val_data = np.column_stack((val_features, val_labels))

In [15]:
np.save("val_data.npy", val_data)

In [16]:
test_features, test_labels = extract_features_dataloader(test_loader, model, device)
test_data = np.column_stack((test_features, test_labels))

In [17]:
np.save("test_data.npy", test_data)